In [4]:
import gym

In [10]:
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random

In [11]:
ENV_NAME = "CartPole-v1"

GAMMA = 0.95
LEARNING_RATE = 0.001

MEMORY_SIZE = 1000000
BATCH_SIZE = 20

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.01
EXPLORATION_DECAY = 0.995

In [12]:

class DQNSolver:

    def __init__(self, observation_space, action_space):
        self.exploration_rate = EXPLORATION_MAX

        self.action_space = action_space
        self.memory = deque(maxlen=MEMORY_SIZE)

        self.model = Sequential()
        self.model.add(Dense(24, input_shape=(observation_space,), activation="relu"))
        self.model.add(Dense(24, activation="relu"))
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() < self.exploration_rate:
            return random.randrange(self.action_space)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def experience_replay(self):
        if len(self.memory) < BATCH_SIZE:
            return
        batch = random.sample(self.memory, BATCH_SIZE)
        for state, action, reward, state_next, terminal in batch:
            q_update = reward
            if not terminal:
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))
            q_values = self.model.predict(state)
            q_values[0][action] = q_update
            self.model.fit(state, q_values, verbose=0)
        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)

In [13]:
env = gym.make(ENV_NAME)
# score_logger = ScoreLogger(ENV_NAME)
observation_space = env.observation_space.shape[0]
action_space = env.action_space.n
dqn_solver = DQNSolver(observation_space, action_space)
run = 0
while True:
    run += 1
    state = env.reset()
    state = np.reshape(state, [1, observation_space])
    step = 0
    while True:
        step += 1
        env.render()
        action = dqn_solver.act(state)
        state_next, reward, terminal, info = env.step(action)
        reward = reward if not terminal else -reward
        state_next = np.reshape(state_next, [1, observation_space])
        dqn_solver.remember(state, action, reward, state_next, terminal)
        state = state_next
        if terminal:
            print("Run: " + str(run) + ", exploration: " + str(dqn_solver.exploration_rate) + ", score: " + str(step))
            # score_logger.add_score(step, run)
            break
        dqn_solver.experience_replay()

2022-05-26 20:05:00.211691: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-26 20:05:00.380071: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Run: 1, exploration: 0.9655206468094844, score: 27
Run: 2, exploration: 0.8348931673187264, score: 30
Run: 3, exploration: 0.7439808620067382, score: 24
Run: 4, exploration: 0.7005493475733617, score: 13
Run: 5, exploration: 0.6696478204705644, score: 10
Run: 6, exploration: 0.6305556603555866, score: 13
Run: 7, exploration: 0.5647174463480732, score: 23
Run: 8, exploration: 0.510849320360386, score: 21
Run: 9, exploration: 0.4858739637363176, score: 11
Run: 10, exploration: 0.46444185833082485, score: 10
Run: 11, exploration: 0.43732904629000013, score: 13
Run: 12, exploration: 0.4159480862733536, score: 11
Run: 13, exploration: 0.37816180712868996, score: 20
Run: 14, exploration: 0.3578751580867638, score: 12
Run: 15, exploration: 0.33362200135903064, score: 15
Run: 16, exploration: 0.3173112652388396, score: 11
Run: 17, exploration: 0.3017979588795719, score: 11
Run: 18, exploration: 0.2913921604631864, score: 8
Run: 19, exploration: 0.27853872940185365, score: 10
Run: 20, explorati

KeyboardInterrupt: 